# Architecture 1: AlexNet Variant

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, GaussianDropout, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, load_model
# from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
# import autokeras as ak
import timeit

## Defining the Model
- A series of incremental smaller convolution kernels feeding through the Convolutional (Conventional 2D) layer:
    - 9 x 9 x 16
    - 7 x 7 x 32
    - 5 x 5 x 64
    - 3 x 3 x 128
- Each Conv2D layer is followed by a batch normalization layer and a ReLU activation layer
- A 2 x 2 max-pooling layer between two Convolutional layers
- GausianDropout at
    - 0.2 rate after each pooling layer
    - 0.5 rate after each dense layer
- flatten
- 2 fully connected Dense layers before the output layer

- Output layer:
    - return the valence/arousal pair, also **regression = True**: input_shape = 2, linear activation
        - compile: mean_squared_error, adam optimizer
        - return 2 floats (valence, arousal)
    - return the Expression classification, also **regression = False**: input_shape = 8, softmax activation
        - compile: categorical_crossentropy, adam optimizer, metrics = ['accuracy']
        - return 1 label (0 to 8)

In [ ]:
# arguments:
#   image_size: assuming all the input photo has size m*m. The default of this dataset is 128
#   R_or_C: regression (0) or classification (1). We are more interested in regression so the default is set to 0
def alexnet_var_model(image_size = 128,
                      regression = True, 
                      conv_shapes = [[16, (9, 9)], [32, (7, 7)], [64, (5, 5)], [128, (3, 3)], [128, (3, 3)]], 
                      dropout=[0.2, 0.5]) :

    model = Sequential()

    # Convolution blocks
    for i in range (len(conv_shapes)) :
        if (i==0) : 
            # the package requires input_shape when Conv2D is the first layer of the model
            model.add(Conv2D(conv_shapes[i][0], conv_shapes[i][1], input_shape=(image_size, image_size, 3), padding='same', use_bias=False))
        else:
            model.add(Conv2D(conv_shapes[i][0], conv_shapes[i][1], padding='same', use_bias=False))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
        model.add(GaussianDropout(dropout[0]))


    # Flatten
    model.add(Flatten())

    # Adding 2 Dense layers
    for i in range (2) :
        model.add(Dense(1024))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(GaussianDropout(dropout[1]))

    
    # Output layer
    if (regression) :      # 0 represents Regression, 1 represents Classification
        model.add(Dense(2, activation='linear'))
        model.compile(optimizer='adam', loss='mean_squared_error', metrics = ['accuracy'])
    else :
        model.add(Dense(10, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics = ['accuracy'])

    return model

## Training Procedure
1. load x (path) and y (label) data 
2. define training batches
3. implement the training model defined above

In [ ]:
class Training_Procedure :
    
    def __init__(self, model, image_size, train_input_path, train_output, test_input_path, test_output, batch_size, epochs, regression=True) :

        self.model = model
        self.image_size = image_size
        
        self.train_input_path = train_input_path
        self.train_output = train_output

        self.test_input_path = test_input_path
        self.test_output = test_output

        self.batch_size = batch_size
        self.epochs = epochs

        self.regression = regression

    # def load_image(path, num_arr) :
    #     images = []
    #     for num in num_arr :
    #         images.append(np.asarray(Image(path + "/" + num + ".jpg")))

    #     return np.asarray(images)


    # have yet known how to load data for regression 
    #   since the label values must be type integers
    def load_train_data(self, sub_set="training", val_split=0.2) :
        train_data = tf.keras.preprocessing.image_dataset_from_directory(
            self.train_input_path,
            labels=list(self.train_output.astype(int)),
            color_mode='rgb',
            # Use 20% data as testing data.
            validation_split=val_split,
            subset=sub_set,
            # Set seed to ensure the same split when loading testing data.
            seed=123,
            image_size=(self.image_size, self.image_size),
            batch_size=self.batch_size,
        )
        return train_data



    def load_test_data(self) :
        test_data = tf.keras.preprocessing.image_dataset_from_directory(
            self.test_input_path,
            labels=list(self.test_output.astype(int)),
            color_mode='rgb',
            # Set seed to ensure the same split when loading testing data.
            seed=123,
            image_size=(self.image_size, self.image_size),
            batch_size=self.batch_size,
        )
        return test_data


    def process_classification(self) :

        train_data = self.load_train_data()
        val_data = self.load_train_data(sub_set="validation")
        test_data = self.load_test_data()

        # Training process
        startTime = timeit.default_timer()
        hist = self.model.fit(train_data,
                        validation_data=val_data,
                        epochs=self.epochs,
                        batch_size=self.batch_size, verbose=1)
        print('Training time:', timeit.default_timer() - startTime)

        # Testing
        startTime = timeit.default_timer()
        accuracy = self.model.evaluate(test_data)
        print('Testing time:', timeit.default_timer() - startTime)

        return hist.history, accuracy, self.model.count_params()


    def process(self) :
        if (not self.regression) : 
            return self.process_classification()

In [ ]:
def main() :
    model = alexnet_var_model()
    train_input_path = "C:/Phanh/BuAnhNet/EAAI23/data/train_set/New Folder"
    train_output = np.load("C:/Phanh/BuAnhNet/EAAI23/data/trainval_class.npy")

    test_input_path = "C:/Phanh/BuAnhNet/EAAI23/data/val_set/New Folder"
    test_output = np.load("C:/Phanh/BuAnhNet/EAAI23/data/test_class_sorted.npy")
    batch_size = 400 
    epochs = 24

    training_procedure = Training_Procedure(
        model, 
        image_size=128, 
        train_input_path=train_input_path, 
        train_output=train_output, 
        test_input_path=test_input_path,
        test_output=test_output,
        batch_size=batch_size,
        epochs=epochs,
        regression=False    
    )

    model.save('AlexNet_Variant_20220227')
    hist = training_procedure.process()

    train_loss = hist.history['loss']
    train_acc = hist.history['accuracy']
    val_loss = hist.history['val_loss']
    val_acc = hist.history['val_accuracy']

    viz_res(train_loss, train_acc, val_loss, val_acc, 'alexnet_var_20220227')



In [ ]:
import os
# len(os.listdir("C:/Phanh/train_set/train_set/images"))
# os.getcwd()

main()

## Result Visualization
Visualize the training and validation loss and accuracy

In [ ]:
def viz_res(trainLoss, trainAcc, valLoss, valAcc, savename = None):
    plt.figure(figsize=(15,7))

    plt.subplot(1, 2, 1) # row 1, col 2 index 1
    plt.plot(trainLoss, color='#17bccf', label='Train')
    plt.plot(valLoss, color='#ff7f44', label='Val')
    plt.title("Loss Func")
    plt.xlabel('Epochs')
    plt.grid()
    plt.legend()

    plt.subplot(1, 2, 2) # index 2
    plt.plot(trainAcc, color='#17bccf', label='Train')
    plt.plot(valAcc, color='#ff7f44', label='Val')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.grid()
    plt.legend()

    plt.tight_layout()
    if savename != None: plt.savefig('./figure/' + savename + '.jpg', dpi=500)
    plt.show()

## Main

In [1]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

tf.test.is_gpu_available()


[]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

In [ ]:
import torch

torch.cuda.is_available()